In [18]:
import numpy as np
np.random.seed(1337)
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline


from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam
from collections import deque
import random
import urllib, json
import re
from datetime import datetime
import requests


In [2]:
memory = deque(maxlen=2000)
gamma = 0.95  
epsilon = 0.1  # exploration rate
epsilon_min = 0.01
epsilon_decay = 0.99
learning_rate = 0.001
batch_size = 32
memory = []
action_hist = []
class Account:
    cash = 100000
    amount_held = 0
    def calc_value(self,current_price):
        return self.amount_held * current_price
account = Account()

In [102]:
def build_network(input_sz):
    model = Sequential()
    model.add(LSTM(64,
               input_shape=(5, input_sz),
               return_sequences=False,
               stateful=False))
    model.add(Dropout(0.5))
    model.add(Dense(125, activation='relu'))
    model.add(Dense(50,activation='relu'))
    model.add(Dense(2, activation='linear'))
    try:
        model.load_weights("rltrader.h5")
        print "LOADEd weights"
    except:
        pass
    model.compile(loss='mse',optimizer=Adam(lr=learning_rate)) 
    return model

In [4]:
def get_data(symbol):
    q = deque(maxlen=3)
    url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&outputsize=full&symbol="+symbol+"&apikey=TOGGSCK5JLEEJTGL"
    url_rsi = "https://www.alphavantage.co/query?function=RSI&symbol="+symbol+"&interval=daily&time_period=50&series_type=close&apikey=TOGGSCK5JLEEJTGL"
    url_sma = "https://www.alphavantage.co/query?function=SMA&symbol="+symbol+"&interval=daily&time_period=50&series_type=close&apikey=TOGGSCK5JLEEJTGL"
    url_cci = "https://www.alphavantage.co/query?function=CCI&symbol="+symbol+"&interval=daily&time_period=50&series_type=close&apikey=TOGGSCK5JLEEJTGL"
    url_macd = "https://www.alphavantage.co/query?function=MACD&symbol="+symbol+"&interval=daily&series_type=open&apikey=TOGGSCK5JLEEJTGL"
    response_prices = requests.get(url)
    response_rsi = requests.get(url_rsi)
    response_sma = requests.get(url_sma)
    response_cci = requests.get(url_cci)
    response_macd = requests.get(url_macd)
    
    data = response_prices.json()
    data_rsi = response_rsi.json()
    data_sma = response_sma.json()
    data_cci = response_cci.json()
    data_macd =  response_macd.json()
    
    
    data = data["Time Series (Daily)"]
#     print(data_rsi)
    data_rsi = data_rsi["Technical Analysis: RSI"]
    data_sma = data_sma["Technical Analysis: SMA"]
    data_cci = data_cci["Technical Analysis: CCI"]
    data_macd = data_macd["Technical Analysis: MACD"]
    
    ds = {}
    ds["date"] = []
    ds["close"] = []
    ds["trend"] = []
#     ds["open"] = []
#     ds["volume"] = []
#     ds["high"] = []
#     ds["low"] = []
    ds["rsi"] = []
    ds["sma"] = []
    ds["cci"] = []
    ds["macd"] = []
    
#     i = 0
    q.append(1)
    for time_point in data:
        try:
            ds["date"] += [time_point]
            ds["close"] += [float(data[time_point]["4. close"])]
            ds["trend"] += [(float(data[time_point]["4. close"])-(sum(list(q))/len(list(q))))/(sum(list(q))/len(list(q)))]
            last_close = float(data[time_point]["4. close"])
            if last_close <= 0:
                q.append(1)
            else:
                q.append(last_close)
#             ds["open"] += [float(data[time_point]["1. open"])]
#             ds["volume"] += [float(data[time_point]["5. volume"])]
#             ds["high"] += [float(data[time_point]["2. high"])]
#             ds["low"] += [float(data[time_point][ "3. low"])]
        except KeyError as e:
            continue
        try:
            ds["macd"] += [float(data_macd[time_point]["MACD"])]
        except KeyError as e:
            ds["macd"] += [sum(ds["macd"])/len(ds["macd"])]
        try:
            ds["rsi"] += [float(data_rsi[time_point]["RSI"])]
        except KeyError as e:
            ds["rsi"] += [sum(ds["rsi"])/len(ds["rsi"])]
        try:
            ds["sma"] += [float(data_sma[time_point]["SMA"])]
        except KeyError as e:
            ds["sma"] += [sum(ds["sma"])/len(ds["sma"])]
        try:
            ds["cci"] += [float(data_cci[time_point]["CCI"])]
        except KeyError as e:
            ds["cci"] += [sum(ds["cci"])/len(ds["cci"])]
    
    df=pd.DataFrame(ds)
    df = df.sort_values(by='date')
    df = df.reset_index(drop=True)
    return df
        


In [5]:
def remember(s,a,r,sp):
    memory.append((s,a,r,sp))

In [114]:
def update(memory):
#     Take random sample from memory
    batch = random.sample(memory, batch_size) 
#     Train network on batch
    for s, a, r, sp in batch:
#       if state is final there is no future rewards
        target = r
#       discounted future rewards
    
        target = r + gamma * \
           np.amax(model.predict(sp)[0])
                
#       Get pred then replace action with our target
        target_f = model.predict(s)
#         print target
#         print target_f
        target_f[0][a] = target
        model.fit(s, target_f, epochs=1, verbose=0)

In [7]:
def get_act(obs):
    
#     if np.random.rand() <= epsilon:
# #         print "random"
# #         # The agent acts randomly
# #         epsilon = epsilon*epsilon_decay
#         act = int(random.uniform(0, 2))
#         return act
#     print "optimal"
    # Predict the reward value based on the given state
    act_values = model.predict(obs)

    # Pick the action based on the predicted reward
    return np.argmax(act_values[0])

In [8]:
int(random.uniform(0, 2))

0

In [98]:

def act(act,day):
    current_val = account.calc_value(day.close.values[0])
    i = day.index[0]
    future = df.close.iloc[i+1:i+3].mean()
    if act == 1:
        buy["buy"] += [day.close.values[0]]
        buy["date"] += [df_d.date.iloc[i]]
        if (account.cash - (day.close.values[0]*5)) >= 0:
            account.amount_held =  account.amount_held + 5
            account.cash = account.cash - (5*day.close.values[0])
        if ((future - day.close.values[0])/day.close.values[0]) >= 0:
            return 1
        else:
            return -1
#             if (account.calc_value(future) - current_val) > 0:
#                 return 1
#             else:
#                 return -1
    #         Buy some shares reward is if we make money in a week
    else:
        
        sell["sell"] += [day.close.values[0]]
        sell["date"] += [df_d.date.iloc[i]]
        to_sell =int((random.random()*account.amount_held))
        if account.amount_held > 0:
            account.cash = account.cash + (to_sell*day.close.values[0])
            account.amount_held = account.amount_held -to_sell
        if -((future - day.close.values[0])/day.close.values[0]) > 0:
#             print "Sell" , (future - day[1]["close"])/day[1]["close"]
            return 1
        else:
            return -1
#         if (account.calc_value(future) - current_val) < 0:
#             return 1
#         else:
#             return -1
    
    return -0.05
#         Sell some shares reward is if we avoid losing money in a week

In [10]:

#     print d[1]["date"]


In [94]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in xrange(0, len(seq), size))

for i in chunker(df,5):
        print i.tail(1)
        print i.tail(1).close.values[0]
        i = i.values
#         print i
        i = i.reshape(1, i.shape[0], i.shape[1])
#         print len(i)

         cci  close      macd        rsi        sma     trend
4  19.834362  41.88  0.270557  52.143318  32.386774  0.228032
41.88
         cci  close      macd        rsi       sma     trend
9  22.345831  43.94  0.190091  51.910715  31.72013 -0.651914
43.94
          cci  close      macd        rsi        sma     trend
14  20.987754  42.75  0.290183  52.563507  32.104338  0.650898
42.75
          cci  close     macd       rsi        sma     trend
19  20.931737  37.06  0.18604  51.92146  32.010068  0.408768
37.06
          cci  close     macd        rsi        sma     trend
24  18.005221  38.06  0.25372  51.885184  33.001733 -0.397181
38.06
          cci  close      macd       rsi        sma     trend
29  22.188627   50.0  0.207495  51.89447  31.780431 -0.019223
50.0
          cci  close    macd        rsi        sma     trend
34  20.847191   65.5  4.3209  52.003551  32.548116  0.224604
65.5
          cci  close    macd        rsi        sma     trend
39  21.569992   64.0  5.3357  51.92

         cci  close    macd      rsi      sma     trend
364  99.0526  93.79  3.7608  56.3261  82.8684 -0.015121
93.79
         cci  close    macd      rsi      sma     trend
369  23.0054  90.01  1.9756  54.5941  85.8502  0.968794
90.01
         cci  close    macd    rsi      sma     trend
374  46.4607  90.85  1.2481  54.59  88.1214  3.064725
90.85
         cci  close   macd      rsi      sma     trend
379 -60.7803  83.26  0.362  51.5795  88.9074  0.437335
83.26
          cci  close    macd      rsi      sma     trend
384 -107.5993  82.67 -2.2668  51.3803  88.7372  0.725047
82.67
          cci  close    macd      rsi      sma     trend
389 -124.2417  77.11 -3.4242  49.4396  87.5112 -0.174146
77.11
          cci  close    macd      rsi      sma     trend
394 -110.4705   79.0 -4.4872  50.2823  86.7914  2.712988
79.0
        cci  close    macd     rsi      sma     trend
399  5.6396  87.88 -1.3346  53.168  86.2474  3.317367
87.88
        cci  close   macd      rsi      sma     trend
404 -6.

       cci  close    macd      rsi      sma     trend
734  60.68  14.11  1.2153  48.6529  11.8338 -0.474031
14.11
         cci  close   macd      rsi      sma     trend
739  44.3417  13.77  0.437  48.3236  12.3334  0.427929
13.77
        cci  close   macd      rsi      sma     trend
744 -1.7467  12.65  0.006  46.9496  12.8836 -0.184046
12.65
        cci  close    macd      rsi      sma    trend
749 -43.083  11.95 -0.2674  46.0379  13.2746 -0.54032
11.95
        cci  close    macd      rsi      sma     trend
754 -16.769  13.31 -0.4636  48.3304  13.4826 -0.299474
13.31
         cci  close    macd      rsi      sma     trend
759 -58.3471  12.27 -0.3279  46.9069  13.6302 -0.469367
12.27
          cci  close    macd      rsi      sma     trend
764 -128.9681  10.48 -0.5295  44.2875  13.3082 -0.722874
10.48
          cci  close    macd     rsi      sma     trend
769 -138.4236  10.38 -0.8003  44.401  13.1524 -0.209544
10.38
          cci  close    macd      rsi      sma     trend
774 -113.6552

          cci  close    macd      rsi      sma     trend
1104 -28.8327   22.5 -0.4258  50.2735  23.1006 -0.190356
22.5
          cci  close    macd      rsi      sma     trend
1109 -29.5853  22.13  0.1464  49.3123  23.1798 -0.500527
22.13
          cci  close  macd      rsi      sma     trend
1114 -87.6601   20.8 -0.04  46.5085  22.8756  0.419472
20.8
           cci  close    macd      rsi      sma     trend
1119 -131.9288  19.76 -0.5464  44.5381  22.2398 -0.082353
19.76
           cci  close    macd      rsi      sma     trend
1124 -102.9086  20.25 -0.6002  46.0156  21.6332 -0.260319
20.25
           cci  close    macd     rsi      sma     trend
1129 -151.8567  19.26 -0.4703  44.165  21.2996 -0.394213
19.26
           cci  close    macd      rsi      sma     trend
1134 -260.9044  16.51 -1.0195  38.9142  20.8206 -0.040488
16.51
           cci  close    macd      rsi      sma     trend
1139 -212.9799  15.35 -1.4422  37.1049  20.2786 -0.443168
15.35
           cci  close    macd      rsi

          cci  close    macd      rsi      sma     trend
1474  87.0105  34.04  0.5486  56.2379  32.8756  0.326923
34.04
           cci  close   macd      rsi      sma    trend
1479  207.8385  36.19  0.735  59.5933  33.2908 -0.65108
36.19
           cci  close    macd     rsi      sma     trend
1484  205.5646  37.88  1.2567  61.841  33.7714 -0.626957
37.88
          cci  close    macd      rsi   sma     trend
1489  91.9383  35.91  0.8673  56.7649  34.1  0.469313
35.91
          cci  close    macd      rsi     sma     trend
1494  98.5964  36.87  0.5557  58.1817  34.372  0.191788
36.87
          cci  close    macd      rsi      sma     trend
1499  71.9379  36.24  0.5593  56.2742  34.6486  0.145929
36.24
           cci  close    macd      rsi      sma     trend
1504  105.7086  37.77  0.4485  58.8302  35.1412 -0.414025
37.77
           cci  close    macd   rsi      sma     trend
1509  146.2742  39.29  0.5422  60.2  35.7426  2.211279
39.29
           cci  close    macd      rsi      sma    t

...

          cci  close    macd      rsi      sma     trend
2954 -39.8599  13.09  0.0814  45.1717  13.6679 -0.391964
13.09
          cci  close    macd      rsi     sma    trend
2959  19.6151  13.89 -0.2919  47.8569  13.503  0.03118
13.89
           cci  close    macd      rsi      sma     trend
2964  136.1027  15.46  0.1141  51.6417  13.5462 -0.301611
15.46
          cci  close    macd      rsi      sma     trend
2969  43.7333  14.41  0.3649  49.0465  13.7896 -0.559686
14.41
          cci  close    macd      rsi      sma    trend
2974  91.2536  15.25  0.2866  51.0448  13.9568 -0.30545
15.25
          cci  close    macd      rsi      sma    trend
2979  12.5007  14.65  0.0951  49.6854  14.1777 -0.28964
14.65
          cci  close   macd      rsi      sma     trend
2984  14.5249  14.47  0.103  49.2741  14.3295 -0.428364
14.47
          cci  close    macd      rsi      sma     trend
2989 -30.2383  14.06  0.0701  48.3126  14.4371 -0.392263
14.06
          cci  close    macd      rsi      sma  

          cci  close    macd      rsi      sma     trend
3319  63.0617  12.64  0.1018  50.4915  12.4673 -0.008109
12.64
         cci  close    macd      rsi      sma     trend
3324  8.7015  12.48  0.0272  49.3747  12.4511 -0.576518
12.48
         cci  close  macd      rsi      sma     trend
3329  8.4471  12.41  0.02  48.9001  12.4929 -0.226164
12.41
          cci  close    macd      rsi      sma     trend
3334 -64.2087  12.43 -0.0697  49.2081  12.5066 -0.407531
12.43
           cci  close    macd      rsi      sma     trend
3339  145.7648  12.82  0.0495  51.7229  12.5506 -0.282195
12.82
         cci  close    macd      rsi      sma    trend
3344  1.7744  12.59  0.0504  50.0724  12.5875 -0.43978
12.59
           cci  close    macd      rsi      sma     trend
3349  274.1602  13.57  0.1902  56.1077  12.6739 -0.327885
13.57
           cci  close    macd      rsi     sma     trend
3354  217.1524  13.83  0.3253  57.2915  12.803 -0.224196
13.83
           cci  close    macd      rsi     sma  

          cci   close    macd      rsi     sma     trend
3684  97.3915  19.455  0.2136  55.2642  18.664  0.014162
19.455
           cci   close    macd      rsi      sma     trend
3689  129.8663  19.685  0.2689  56.1426  18.6894 -0.007396
19.685
          cci  close    macd      rsi     sma     trend
3694  92.3851  19.61  0.3146  55.1233  18.822  0.019672
19.61
          cci  close    macd      rsi      sma     trend
3699  35.8833  19.15  0.1135  52.3669  18.8649 -0.122365
19.15
         cci  close    macd      rsi      sma     trend
3704  3.6366  18.92  0.0124  50.9647  18.8512 -0.755621
18.92
         cci  close    macd      rsi      sma     trend
3709 -54.441  18.27 -0.1634  47.5859  18.8148  0.515344
18.27
          cci  close    macd      rsi      sma     trend
3714 -89.8906  18.26 -0.2887  47.7928  18.8397  0.009304
18.26
           cci  close    macd      rsi      sma     trend
3719 -175.4103  17.44 -0.4998  44.6662  18.7912 -0.262891
17.44
          cci  close    macd      rsi 

          cci  close   macd      rsi      sma     trend
4054 -95.2565  26.99 -1.255  47.2503  30.1922  0.058985
26.99
          cci  close    macd      rsi      sma     trend
4059  46.9363   31.6  0.0488  55.9702  29.9062  0.309573
31.6
          cci  close    macd      rsi      sma     trend
4064  75.6397  32.75  0.7813  57.6618  29.8126  0.001121
32.75
         cci  close    macd      rsi      sma     trend
4069  64.583  31.75  1.0466  55.2679  29.7834  1.066161
31.75
          cci  close    macd     rsi     sma     trend
4074  78.1428  32.14  0.8388  55.877  29.658  0.394562
32.14
           cci  close    macd      rsi      sma     trend
4079  119.7761  33.85  0.9315  58.7526  29.8738  0.626622
33.85
           cci  close    macd      rsi      sma     trend
4084  143.0838  35.76  1.1599  62.0494  30.4736 -0.176227
35.76
           cci  close    macd      rsi      sma     trend
4089  108.6674   35.8  1.2473  61.5514  31.2666 -0.073259
35.8
           cci  close    macd      rsi      

          cci   close    macd      rsi       sma     trend
4424  97.5798  166.48  3.7989  61.5368  154.3748  4.546794
166.48
          cci   close    macd      rsi       sma     trend
4429  87.4737  164.74  4.2499  59.2798  157.1512  3.038735
164.74
         cci   close    macd      rsi      sma     trend
4434  37.867  161.47  2.2081  56.4537  158.861  4.755138
161.47
          cci   close    macd      rsi       sma     trend
4439  57.8142  165.19  0.9389  57.3309  159.8922  1.324982
165.19
          cci   close    macd      rsi       sma    trend
4444  80.1848  169.44  0.8668  58.5271  160.9068  4.19489
169.44
          cci   close    macd      rsi       sma      trend
4449  27.2224  163.69  1.1674  55.7954  162.3022  10.524759
163.69
           cci  close    macd      rsi       sma  trend
4453  106.6739  169.4  1.3622  57.5692  164.3974  8.075
169.4


In [ ]:
# model = build_network(len(df.columns))
# prv_obs = None
feats = ["trend","rsi","cci","macd"]
buy = {}
buy["buy"] = []
buy["date"] = []
sell = {}
sell["sell"] = []
sell["date"] = []
worth = {}
worth["worth"]=[]
gl = []
epochs = 20
df_d = None
codes = ["TSLA","MSFT","ASX:ANZ","NVDA"]
for epoch in range(0,epochs):
    epsilon = epsilon*((epochs-epoch)/epochs)
    buy = {}
    buy["buy"] = []
    buy["date"] = []
    sell = {}
    sell["sell"] = []
    sell["date"] = []
    worth = {}
    worth["worth"]=[]
    gl = []
    i = 0
    memory = deque(maxlen=2000)
    account = Account()
    lets_trade = random.choice(codes)
#     lets_trade = "TSLA"
    print lets_trade
    df = get_data(lets_trade)
    df_d = df[["date","close"]]
    del df["date"]
    model = build_network(len(df[feats].columns))
    prv_obs = None

    for seq in chunker(df,5):
        i = 1 + i
        if i == len(df) - 11 or len(seq) != 5:
            model.save_weights("rltrader.h5")
            print("Episode finished after {} timesteps".format(i+1))
            break
            
#         Convert to LSTM formart (batch,timesteps,cols)
        day = seq.tail(1)
#         print day
        seq = seq[feats].values
        seq = seq.reshape(1, seq.shape[0], seq.shape[1])
        
      
        obs = seq
#         obs_values = np.reshape(obs[feats].values,[1,1,len(df[feats].columns)])
        action = get_act(obs)
        reward = act(action,day)
        if action == 1:
            action_hist += ["buy"] 
        else:
            action_hist += ["sell"]
        if i % 100 == 0:
            print reward
            if action == 1:
                print "Buying" 
            else:
                print "Selling"
            print "day" , i
            print "cash",account.cash
            print "shares held",account.amount_held
            print "net worth",account.calc_value(day.close.values[0]) + account.cash
        worth["worth"] += [account.calc_value(day.close.values[0]) + account.cash]
    #     print(action)
        reward = act(action,day)
    #     print reward
        gl += [(account.calc_value(day.close.values[0]) + account.cash) - 100000]
        if prv_obs is not None and reward is not None:
#             print action
            remember(prv_obs,action,reward,obs)
        prv_obs = obs 
    #     if done:
    #         model.save_weights("model.h5")
    #         print("Episode finished after {} timesteps".format(t+1))
    #         break
        if len(memory) >= batch_size:
            update(memory)
    print ((sum(gl)/len(gl))/100000)*100

MSFT
LOADEd weights
-1
Selling
day 100
cash 89124.07
shares held 1
net worth 89190.32
1
Buying
day 200
cash 86169.96
shares held 76
net worth 88225.0
-1
Buying
day 300
cash 87328.62
shares held 47
net worth 88589.63
-1
Buying
day 400
cash 86059.9106
shares held 76
net worth 88743.4706


In [ ]:
print ((sum(gl)/len(gl))/100000)*100
# df_d
print (133605.115/100000)*100
# print action_hist

In [ ]:
df_buys = pd.DataFrame(buy)
df_sells = pd.DataFrame(sell)
df_worth = pd.DataFrame(worth)

In [ ]:
df_d.plot()
df_worth.plot()


In [ ]:
df_buys.plot(style=".")

In [ ]:
# df_d.plot()
df_sells.plot(style=".")

In [ ]:
df_d.date = pd.to_datetime(df_d.date,format='%Y-%m-%d')
df_d = df_d.set_index("date")
df_buys.date = pd.to_datetime(df_buys.date,format='%Y-%m-%d')
df_buys = df_buys.set_index("date")
df_sells.date = pd.to_datetime(df_sells.date,format='%Y-%m-%d')
df_sells = df_sells.set_index("date")
ax = df_d.plot()

ax = df_d.plot()
ax = df_buys.plot(ax=ax)
df_sells.plot(ax=ax)
# # df_sells.groupby(df_sells["date"])
plt.show()

In [ ]:
fig = plt.figure(figsize = (50,50))
ax = fig.gca()
df.rsi.plot(ax=ax)

In [ ]:
fig = plt.figure(figsize = (50,50))
ax = fig.gca()
ax = df_d.plot(ax=ax)
ax = df_buys.plot(style=".",ax=ax)
df_sells.plot(style=".",ax=ax)
# # df_sells.groupby(df_sells["date"])
plt.show()

In [ ]:
df_d

In [ ]:
%%bash
which python